TABLEAU_PROJECT

In [1]:
import pandas as pd 
import pymongo
MongoClient = pymongo.MongoClient

import geojson
from geojson import Point, Feature, FeatureCollection, dump
import time

client = MongoClient()
db = client.companies

In [2]:
client.companies

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [3]:
cursor = db.companies.find({"$and": [ {"category_code":"software"} , {"founded_year" : {"$gte": 2000} }, {"offices.latitude" : {"$ne": None} }, {"offices.longitude" : {"$ne": None} } ] },{"name": 1, "company_code": 1,"offices.latitude":1, "offices.longitude":1})
ls_cursor= list(cursor)

#print(ls_cursor)

a = []
counter = 0

for i in ls_cursor:
    i = counter
    a.append(ls_cursor[i])
    counter += 1

In [4]:
df = pd.DataFrame(a)
df.head()

,_id,name,offices
0,52cdef7c4bab8bd675297d8c,Zoho,"[{'latitude': 37.692934, 'longitude': -121.904..."
1,52cdef7c4bab8bd675297e30,GoingOn,"[{'latitude': 37.782263, 'longitude': -122.392..."
2,52cdef7c4bab8bd675297e3e,spigit,"[{'latitude': 37.6637279, 'longitude': -121.87..."
3,52cdef7c4bab8bd675297e3c,Bazaarvoice,"[{'latitude': 30.407545, 'longitude': -97.7176..."
4,52cdef7c4bab8bd675297e57,GoLightly,"[{'latitude': 37.898575, 'longitude': -122.537..."


In [5]:
b = []
counter = 0

for i in df["offices"]:
    i = counter
    b.append(df["offices"][i])
    counter += 1

df_b = pd.DataFrame(b)
df_b = df_b[0] # nos quedamos sólo con la primera sucursal
df_b = pd.Series.to_frame(df_b) # convertimos a DF
df['coords'] = df_b
df.head()

,_id,name,offices,coords
0,52cdef7c4bab8bd675297d8c,Zoho,"[{'latitude': 37.692934, 'longitude': -121.904...","{'latitude': 37.692934, 'longitude': -121.904945}"
1,52cdef7c4bab8bd675297e30,GoingOn,"[{'latitude': 37.782263, 'longitude': -122.392...","{'latitude': 37.782263, 'longitude': -122.392142}"
2,52cdef7c4bab8bd675297e3e,spigit,"[{'latitude': 37.6637279, 'longitude': -121.87...","{'latitude': 37.6637279, 'longitude': -121.873..."
3,52cdef7c4bab8bd675297e3c,Bazaarvoice,"[{'latitude': 30.407545, 'longitude': -97.7176...","{'latitude': 30.407545, 'longitude': -97.717667}"
4,52cdef7c4bab8bd675297e57,GoLightly,"[{'latitude': 37.898575, 'longitude': -122.537...","{'latitude': 37.898575, 'longitude': -122.537302}"


In [6]:
# eliminas columna offices
df = df.drop('offices', axis=1, inplace=True)

# eliminas nulos en coors
df 

# json_normalize cords


In [7]:
# df_b = df_b.dropna() # eliminamos valores nuos
df_b.head()

,0
0,"{'latitude': 37.692934, 'longitude': -121.904945}"
1,"{'latitude': 37.782263, 'longitude': -122.392142}"
2,"{'latitude': 37.6637279, 'longitude': -121.873..."
3,"{'latitude': 30.407545, 'longitude': -97.717667}"
4,"{'latitude': 37.898575, 'longitude': -122.537302}"


In [11]:
df

In [ ]:
df_b = df_b.reset_index(drop=True)


In [ ]:
df_b.head()

In [ ]:
df_b[0][65]['longitude']

In [ ]:
df_b.head()

In [ ]:
longitude = []
latitude = []
counter = 0

for i in df_b[0]:
    i = counter
    longitude.append(df_b[0][i]['longitude'])
    latitude.append(df_b[0][i]['latitude'])
    counter += 1


In [ ]:
df_b['lat'] = latitude
df_b['long'] = longitude

df_b.head()

clean_df = pd.concat([df,df_b],axis=1)
clean_df.drop(['offices"], inplace=True, axis=1)


{
   db.places.find : {
     near: {
       geometry: {
          type: "Point" ,
          coordinates: [ longitude , latitude ]
       },
     }
   }
}